# test-via-streaming.ipynb

Test the API implementation of the "via_streaming"  options which tries to get around timeouts by retrieving the response data 
using streaming. However this does not prevent timeouts when the first response already takes too long.

In [8]:
import os, sys
from loguru import logger
from typing import Optional, List, Dict
sys.path.append(os.path.join(".."))
from llms_wrapper.llms import LLMS
from llms_wrapper.config import update_llm_config

In [9]:
config = dict(
    llms=[
        # OpenAI
        # https://platform.openai.com/docs/models
        dict(llm="openai/gpt4"),
        # https://ai.google.dev/gemini-api/docs/models/gemini
        dict(llm="gemini/gemini-2.0-flash"),
        # Anthropic
        # https://docs.anthropic.com/en/docs/about-claude/models
        dict(llm="anthropic/claude-3-5-sonnet-20240620"),
        dict(llm="anthropic/claude-3-opus-20240229"),
        # Mistral
        # https://docs.mistral.ai/getting-started/models/models_overview/
        dict(llm="mistral/mistral-large-latest"),
        # XAI
        # dict(llm="xai/grok-2"),     # not mapped by litellm yet?
        dict(llm="xai/grok-beta"),
        # Groq
        # https://console.groq.com/docs/models
        dict(llm="groq/llama3-70b-8192"),
        dict(llm="groq/llama-3.3-70b-versatile"),
        # Deepseek
        # https://api-docs.deepseek.com/quick_start/pricing
        dict(llm="deepseek/deepseek-chat"),
    ],
    providers = dict(
        openai = dict(api_key_env="MY_OPENAI_API_KEY"),
        gemini = dict(api_key_env="MY_GEMINI_API_KEY"),
        anthropic = dict(api_key_env="MY_ANTHROPIC_API_KEY"),
        mistral = dict(api_key_env="MY_MISTRAL_API_KEY"),
        xai = dict(api_key_env="MY_XAI_API_KEY"),    
        groq = dict(api_key_env="MY_GROQ_API_KEY"),
        deepseek = dict(api_key_env="MY_DEEPSEEK_API_KEY"),
    )
)
config = update_llm_config(config)
llms = LLMS(config)
llms.list_aliases()

['openai/gpt4',
 'gemini/gemini-2.0-flash',
 'anthropic/claude-3-5-sonnet-20240620',
 'anthropic/claude-3-opus-20240229',
 'mistral/mistral-large-latest',
 'xai/grok-beta',
 'groq/llama3-70b-8192',
 'groq/llama-3.3-70b-versatile',
 'deepseek/deepseek-chat']

## Test via_streaming versus default



In [10]:
msgs = LLMS.make_messages("What is a monoid? Give me a simple example. Explain why the concept is useful.")
msgs

[{'content': 'What is a monoid? Give me a simple example. Explain why the concept is useful.',
  'role': 'user'}]

In [13]:
# llmname = "gemini/gemini-2.0-flash"
llmname = "mistral/mistral-large-latest"

In [14]:
# logger.enable("llms_wrapper")
logger.disable("llms_wrapper")
ret = llms.query(
    llmname, msgs, temperature=0.5, litellm_debug=False, 
    via_streaming=True,   # or from config!!
    debug=False,
)
if ret["ok"]:
    print("PROCESSING OK, chunks:", ret["n_chunks"])
    print(ret["answer"].strip())
else:
    print("!!!!!Error:", ret["error"])

PROCESSING OK, chunks: 390
### **What is a Monoid?**
A **monoid** is a simple algebraic structure in mathematics and computer science that consists of:
1. A **set** (e.g., numbers, strings, lists).
2. An **associative binary operation** (e.g., addition, concatenation) that combines two elements of the set.
3. An **identity element** (e.g., `0` for addition, `""` for string concatenation) that leaves other elements unchanged when combined.

Formally, a monoid is a tuple `(M, ✱, e)` where:
- `M` is a set,
- `✱: M × M → M` is an associative operation,
- `e ∈ M` is the identity element such that for any `a ∈ M`:
  - `e ✱ a = a ✱ e = a`.

---

### **Simple Example: Strings with Concatenation**
- **Set (`M`)**: All possible strings (e.g., `"hello"`, `"world"`, `""`).
- **Operation (`✱`)**: String concatenation (`++`).
  - `"a" ++ "b" = "ab"` (associative: `("a" ++ "b") ++ "c" = "a" ++ ("b" ++ "c")`).
- **Identity (`e`)**: The empty string `""`.
  - `"" ++ "x" = "x" ++ "" = "x"`.

This forms 

In [10]:
# Standard/default way to do it .... 
logger.disable("llms_wrapper")
ret = llms.query(
    llmname, msgs, temperature=0.5, litellm_debug=False, debug=False)
if ret["ok"]:
    print("PROCESSING OK, chunks:", ret["n_chunks"])
    print(ret["answer"].strip())
else:
    print("!!!!!Error:", ret["error"])

PROCESSING OK, chunks: 1
A **monoid** is a fundamental algebraic structure in mathematics and computer science that consists of:

1. **A set** (e.g., numbers, strings, functions).
2. **An associative binary operation** (e.g., addition, concatenation) that combines two elements of the set to produce another element of the same set.
3. **An identity element** (e.g., 0 for addition, the empty string for concatenation) that leaves other elements unchanged when combined with them.

### Simple Example: Strings with Concatenation
- **Set**: All possible strings (e.g., `"a"`, `"hello"`, `""`).
- **Operation**: Concatenation (e.g., `"a" + "b" = "ab"`).
- **Identity element**: The empty string `""` (since `"a" + "" = "a"`).

This forms a monoid because:
- Concatenation is **associative**: `(x + y) + z = x + (y + z)`.
- The empty string is the **identity**: `x + "" = x`.

---

### Why Monoids Are Useful
1. **Abstraction**: Monoids generalize common operations (like addition, multiplication, or li